In [1]:
import pickle  # pickle 모듈을 import하여 데이터를 바이너리 형식으로 저장 및 로드할 수 있도록 함

with open('./res/rag-custom.pkl', 'rb') as f:  # 바이너리 읽기 모드로 'rag_data.pkl' 파일 열기
    rag_data = pickle.load(f)  # 파일에서 데이터를 읽어와 'rag_data' 변수에 저장

In [2]:
rag_data['questions'][0]  # 'rag_data'에서 첫 번째 질문을 출력

'What are some of the skills taught in the Trail Patrol Training course?'

In [3]:
rag_data['contexts'][0]  # 'rag_data'에서 첫 번째 문맥 리스트를 출력

'Trail Patrol Training\nWant to be a part of the Trail Patrol ?? Join an Orientation & Hike on the 1st Tuesday of each month. This course is required for all PATC members interested in joining the PATC Trail Patrol.\nThe course teaches the essential skills necessary to be a trail patrol member and to provide a reassuring presence on the trail while teaching safety and environmental responsibility. A Trail Patrol handbook is provided to all students. Please bring a pencil, your hiking daypack & lunch.\nMore Info: View the Calendar or contact TP Training or visit the Trail Patrol Training web pages.\nHike Leader Class.\nMore Info: Contact Hike Leader Training or click here to register.\nBackpacking Classes\nEducating people in safe and environmentally friendly practices for traveling into the backcountry is one of Trail Patrol’s core responsibilities. We offer backpacking classes for novices seeking to take up backpacking as well as for experienced backpackers.\nBackpacking 101: An Intro

In [4]:
from utils import get_embedding, cosine_similarity  # 사용자 정의 모듈 utils에서 get_embedding 함수와 cosine_similarity 함수를 import

embed_q = get_embedding(rag_data['questions'][0])  # 첫 번째 질문의 임베딩을 생성
embed_c0 = get_embedding(rag_data['contexts'][0][0])  # 첫 번째 문맥 리스트의 첫 번째 문맥에 대한 임베딩 생성
embed_c1 = get_embedding(rag_data['contexts'][0][1])  # 첫 번째 문맥 리스트의 두 번째 문맥에 대한 임베딩 생성
embed_c2 = get_embedding(rag_data['contexts'][0][2])  # 첫 번째 문맥 리스트의 세 번째 문맥에 대한 임베딩 생성

print(cosine_similarity(embed_q, embed_c0))  # 질문 임베딩과 첫 번째 문맥 임베딩 간의 코사인 유사도 출력
print(cosine_similarity(embed_q, embed_c1))  # 질문 임베딩과 두 번째 문맥 임베딩 간의 코사인 유사도 출력
print(cosine_similarity(embed_q, embed_c2))  # 질문 임베딩과 세 번째 문맥 임베딩 간의 코사인 유사도 출력

0.11002266520189832
0.07480343102199662
0.041763653646215856


In [5]:
rag_data['contexts_answer_idx'][0]  # 첫 번째 질문에 대한 정답 문맥 인덱스 출력

0

In [6]:
embed_q = get_embedding(rag_data['questions'][0], model='text-embedding-3-large')  # 'text-embedding-3-large' 모델을 사용하여 첫 번째 질문 임베딩 생성
embed_c0 = get_embedding(rag_data['contexts'][0][0], model='text-embedding-3-large')  # 동일한 모델을 사용하여 첫 번째 문맥 임베딩 생성
embed_c1 = get_embedding(rag_data['contexts'][0][1], model='text-embedding-3-large')  # 동일한 모델을 사용하여 두 번째 문맥 임베딩 생성
embed_c2 = get_embedding(rag_data['contexts'][0][2], model='text-embedding-3-large')  # 동일한 모델을 사용하여 세 번째 문맥 임베딩 생성

print(cosine_similarity(embed_q, embed_c0))  # 질문 임베딩과 첫 번째 문맥 임베딩 간의 코사인 유사도 출력
print(cosine_similarity(embed_q, embed_c1))  # 질문 임베딩과 두 번째 문맥 임베딩 간의 코사인 유사도 출력
print(cosine_similarity(embed_q, embed_c2))  # 질문 임베딩과 세 번째 문맥 임베딩 간의 코사인 유사도 출력

0.10867087813080151
0.021356636340793884
0.035892125816805086


In [7]:
from tqdm import tqdm  # 진행 상태 표시를 위한 tqdm 모듈 import

num_questions = 10  # 처리할 질문 수 설정
num_contexts = 3  # 처리할 문맥 수 설정

top_context_indices = []  # 상위 문맥 인덱스를 저장할 리스트 초기화

for i in tqdm(range(num_questions)):  # 질문 수 만큼 반복, 진행 상태를 tqdm으로 표시
  embed_q = get_embedding(rag_data['questions'][i])  # 현재 질문에 대한 임베딩 생성

  similarities = []  # 유사도 값 저장할 리스트 초기화
  for j in range(num_contexts):  # 문맥 수 만큼 반복
    embed_c = get_embedding(rag_data['contexts'][i][j])  # 현재 문맥에 대한 임베딩 생성
    similarities.append(cosine_similarity(embed_q, embed_c))  # 질문과 문맥 간의 코사인 유사도 계산 후 추가

  top_context_index = similarities.index(max(similarities))  # 가장 높은 유사도를 가진 문맥의 인덱스 찾기
  top_context_indices.append(top_context_index)  # 상위 문맥 인덱스 리스트에 추가

print(f"Top context indices: {top_context_indices}")  # 상위 문맥 인덱스 출력

100%|██████████| 10/10 [00:25<00:00,  2.52s/it]

Top context indices: [0, 1, 1, 1, 0, 0, 2, 1, 0, 0]


In [8]:
rag_data['contexts_answer_idx'][:num_questions]  # 정답 문맥 인덱스의 상위 10개 출력

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
def calculate_accuracy(predicted, actual):  # 예측과 실제 정답 간의 정확도를 계산하는 함수 정의
  correct = sum(p == a for p, a in zip(predicted, actual))  # 예측이 실제 정답과 일치하는 경우 수를 계산
  total = len(predicted)  # 총 예측 수 계산

  accuracy = correct / total  # 정확도 계산
  return accuracy  # 정확도 반환

accuracy = calculate_accuracy(top_context_indices, rag_data['contexts_answer_idx'][:10])  # 상위 문맥 인덱스와 정답 문맥 인덱스 비교하여 정확도 계산
print(f"Accuracy: {accuracy:.2%}")  # 정확도를 백분율 형식으로 출력

Accuracy: 50.00%


In [10]:
contexts_predictions = []  # 예측된 문맥 리스트 초기화
for i in range(len(top_context_indices)):  # 예측된 인덱스 수 만큼 반복
    index = top_context_indices[i]  # 현재 예측된 인덱스
    contexts_predictions.append([rag_data['contexts'][i][index]])  # 예측된 문맥 추가
contexts_predictions  # 예측된 문맥 리스트 출력

[['T'], ['o'], ['u'], ['n'], ['('], ['H'], ['i'], ['e'], ['T'], ['S']]

In [11]:
print(contexts_predictions)  # 예측된 문맥 출력

[['T'], ['o'], ['u'], ['n'], ['('], ['H'], ['i'], ['e'], ['T'], ['S']]


In [12]:
print(rag_data['questions'][:10])  # 상위 10개의 질문 출력

['What are some of the skills taught in the Trail Patrol Training course?', 'Who was the original owner of the lot of items being sold?', 'What is the main objective of Humanity Road as a non-profit organization?', 'Who were the two convicted killers that escaped from an upstate New York maximum-security prison?', 'Who was the person that came to help when Isaac was sick?', 'What change has been observed in the banking industry in Hong Kong since the 2008 financial crisis?', 'What is a "Cultural Muslim" according to Kaighla Um Dayo?', "Who was providing financial support for Lou's daughter Joan?", 'What mistakes did the narrator make during his run?', 'Who is the 65th attorney general of New York state?']


In [13]:
print(rag_data['answers'][:10])  # 상위 10개의 정답 출력

['The course teaches the essential skills necessary to be a trail patrol member and to provide a reassuring presence on the trail while teaching safety and environmental responsibility.', 'The original lot of items belonged to Lt. Neil D. Kennedy, an AAF officer who served in the CBI theater.', "The main objective of Humanity Road is to 'close the black hole of communication' by linking disaster affected communities to the emergency response organizations.", 'The two convicted killers that escaped from an upstate New York maximum-security prison were Richard Matt and David Sweat.', 'Jesus was the person who came to help when Isaac was sick.', 'Since the 2008 financial crisis, the banking industry in Hong Kong has become less exciting, more scrutinized, and often less lucrative. As a result, many executives have started their own businesses. Banks are less generous about compensation, have reined in their spending and growth plans, and are not encouraging bankers to innovate.', 'A "Cult

In [14]:
print(rag_data['contexts'][:10])  # 상위 10개의 문맥 리스트 출력

['Trail Patrol Training\nWant to be a part of the Trail Patrol ?? Join an Orientation & Hike on the 1st Tuesday of each month. This course is required for all PATC members interested in joining the PATC Trail Patrol.\nThe course teaches the essential skills necessary to be a trail patrol member and to provide a reassuring presence on the trail while teaching safety and environmental responsibility. A Trail Patrol handbook is provided to all students. Please bring a pencil, your hiking daypack & lunch.\nMore Info: View the Calendar or contact TP Training or visit the Trail Patrol Training web pages.\nHike Leader Class.\nMore Info: Contact Hike Leader Training or click here to register.\nBackpacking Classes\nEducating people in safe and environmentally friendly practices for traveling into the backcountry is one of Trail Patrol’s core responsibilities. We offer backpacking classes for novices seeking to take up backpacking as well as for experienced backpackers.\nBackpacking 101: An Intr

In [16]:
from ragas.llms import LangchainLLMWrapper  # Langchain LLM을 감싸는 래퍼 클래스 import
from langchain_openai import ChatOpenAI  # OpenAI와의 연동을 위한 Langchain 기반 클래스 import
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))  # GPT-4o 모델을 사용한 평가 LLM 인스턴스 생성

In [19]:
from ragas.metrics import LLMContextRecall  # LLMContextRecall 메트릭 import
from ragas import evaluate  # 평가 함수 import
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset  # 평가 데이터셋 스키마와 단일 회차 샘플 import

metrics = [
    LLMContextRecall(llm=evaluator_llm),  # 평가를 위한 LLMContextRecall 메트릭 리스트로 추가
]

samples = []  # 평가 샘플 리스트 초기화
for question, context, answer in zip(rag_data['questions'][:num_questions], contexts_predictions, rag_data['answers'][:num_questions]):  # 질문, 예측 문맥, 정답을 묶어서 반복
    sample = SingleTurnSample(
        user_input=question,  # 사용자 입력 설정
        reference=answer,  # 참조 정답 설정
        retrieved_contexts=context  # 예측된 문맥 설정
    )
    samples.append(sample)  # 샘플 리스트에 추가
dataset = EvaluationDataset(samples)  # 평가 데이터셋 생성

score = evaluate(dataset, metrics=metrics)  # 데이터셋과 메트릭을 사용해 평가 실행
score  # 평가 점수 출력

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

{'context_recall': 0.0000}